# Necessary Tools and Services

In [32]:
# ! pip install pandas
# ! pip install matplotlib
# ! pip install numpy
# ! pip install seaborn
# ! pip install unzip
# ! pip install gensim
# ! pip install nltk
# ! pip install wordcloud
# ! pip install spacy
# ! pip install spacy_download
# ! pip install pyLDAvis
# ! pip install PyStemmer

# ! python3 -m spacy download en

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
def FindingPostsWithNegativeSentiments():
    df = pd.read_csv('../dataset/SO_Workflow_Data.csv')
    my_value = -1
    data = df.loc[df["RatingsGPTFineTuned"] == my_value]

    df = data[data['Body'].str.contains('rapidminer', case=False)] 
    print(len(df))
    df.to_csv('../dataset/rapidminer_posts.csv', index=False)

FindingPostsWithNegativeSentiments()

71


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/rapidminer_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Title","Body"]].apply("-".join, axis=1)
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

71


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['RapidMiner error: Regular Attributes must be of type binomial. Market Basket '
 'Analysis-I am attempting to learn to use RapidMiner, and my boss wants me to '
 'perform a market basket analysis on a set of data. But when I use the given '
 'template, I get the following error: Regular Attributes must be of type '
 'binomial. This is given withing the FP-Growth operator. I have a customerID '
 '(only numbers), a productName(Letters) and a Product Quantity (numbers) '
 'column. As I am a newbie with RM, I have no idea what is wrong. Any input '
 'would be greatly appreciated. Thank you in advance. ',
 'rapidminer some concepts-In rapidminer what is the meaning of: - training '
 'cycles - learning rate - momentum Im trying to work with rapidminer but i '
 'cant understand this notions. Any help would be apreciated, as newbly as '
 'possible. ']


# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['rapidminer', 'error', 'regular', 'attributes', 'must', 'be', 'of', 'type', 'binomial', 'market', 'basket', 'analysis', 'am', 'attempting', 'to', 'learn', 'to', 'use', 'rapidminer', 'and', 'my', 'boss', 'wants', 'me', 'to', 'perform', 'market', 'basket', 'analysis', 'on', 'set', 'of', 'data', 'but', 'when', 'use', 'the', 'given', 'template', 'get', 'the', 'following', 'error', 'regular', 'attributes', 'must', 'be', 'of', 'type', 'binomial', 'this', 'is', 'given', 'withing', 'the', 'fp', 'growth', 'operator', 'have', 'customerid', 'only', 'numbers', 'productname', 'letters', 'and', 'product', 'quantity', 'numbers', 'column', 'as', 'am', 'newbie', 'with', 'rm', 'have', 'no', 'idea', 'what', 'is', 'wrong', 'any', 'input', 'would', 'be', 'greatly', 'appreciated', 'thank', 'you', 'in', 'advance']]


In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['rapidminer', 'error', 'regular', 'attribute', 'type', 'binomial', 'market', 'basket', 'analysis', 'attempt', 'learn', 'use', 'rapidminer', 'boss', 'want', 'perform', 'market', 'basket', 'analysis', 'set', 'data', 'use', 'give', 'template', 'get', 'follow', 'error', 'regular', 'attribute', 'type', 'binomial', 'give', 'withe', 'fp_growth', 'operator', 'customerid', 'number', 'productname', 'letter', 'product', 'quantity', 'number', 'wrong', 'input', 'greatly', 'appreciate', 'advance']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 2), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 2), (28, 1), (29, 1), (30, 2), (31, 2), (32, 1), (33, 1), (34, 1)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('advance', 1),
  ('analysis', 2),
  ('appreciate', 1),
  ('attempt', 1),
  ('attribute', 2),
  ('basket', 2),
  ('binomial', 2),
  ('boss', 1),
  ('customerid', 1),
  ('data', 1),
  ('error', 2),
  ('follow', 1),
  ('fp_growth', 1),
  ('get', 1),
  ('give', 2),
  ('greatly', 1),
  ('input', 1),
  ('learn', 1),
  ('letter', 1),
  ('market', 2),
  ('number', 2),
  ('operator', 1),
  ('perform', 1),
  ('product', 1),
  ('productname', 1),
  ('quantity', 1),
  ('rapidminer', 2),
  ('regular', 2),
  ('set', 1),
  ('template', 1),
  ('type', 2),
  ('use', 2),
  ('want', 1),
  ('withe', 1),
  ('wrong', 1)]]

In [11]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [12]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [06:43<00:00,  1.34it/s]


In [13]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.545106
358,100% Corpus,4,asymmetric,0.9099999999999999,0.530929
523,100% Corpus,10,0.61,0.9099999999999999,0.530743
417,100% Corpus,6,asymmetric,0.61,0.524255
418,100% Corpus,6,asymmetric,0.9099999999999999,0.520599
...,...,...,...,...,...
301,100% Corpus,3,0.01,0.31,0.333287
304,100% Corpus,3,0.01,symmetric,0.333239
326,100% Corpus,3,asymmetric,0.31,0.325645
327,100% Corpus,3,asymmetric,0.61,0.325645


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.91,
                                           eta=0.91)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -6.655919755909586

Coherence Score:  0.5174818514521227


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.108390 -0.019273       1        1  34.955843
9     -0.018619  0.046129       2        1  18.792968
5      0.006890  0.031627       3        1  12.975637
6      0.004303  0.027589       4        1  12.301425
3      0.014477 -0.018528       5        1   6.225812
2      0.015610 -0.029455       6        1   5.066687
0      0.016956 -0.006378       7        1   3.663568
4      0.021411 -0.008989       8        1   2.104988
8      0.023978 -0.011463       9        1   2.086882
1      0.023383 -0.011259      10        1   1.826189, topic_info=        Term       Freq      Total Category  logprob  loglift
437   column  23.000000  23.000000  Default  30.0000  30.0000
105     name  16.000000  16.000000  Default  29.0000  29.0000
31       use  45.000000  45.000000  Default  28.0000  28.0000
62      java  42.000000  42.000000  Default  27.0000  27.0000
192      com  61.000000  61.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
133     good   0.181657   4.736973  Topic10  -6.3516   0.7419
136  instead   0.161108   4.269246  Topic10  -6.4717   0.7258
63      know   0.161808   5.012092  Topic10  -6.4673   0.5697
124    cause   0.159234   4.426735  Topic10  -6.4834   0.6779
147    space   0.159636   7.958759  Topic10  -6.4808   0.0938

[493 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
674       1  0.676768      activated_true
674       3  0.135354      activated_true
153       1  0.373337            adaboost
120       1  0.274020             address
120       6  0.274020             address
...     ...       ...                 ...
1139      1  0.252551                yarn
1139      6  0.505102                yarn
1140      1  0.342270  yarn_resourcemanag
1140      6  0.342270  yarn_resourcemanag
722       1  0.382619           zoomlocke

[914 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 6, 7, 4, 3, 1, 5, 9, 2])

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.006*"class" + 0.005*"detail" + 0.005*"get" + 0.005*"firebase" + '
  '0.005*"try" + 0.005*"use" + 0.004*"span" + 0.004*"error" + 0.004*"m" + '
  '0.004*"block"'),
 (1,
  '0.004*"export" + 0.002*"issue" + 0.002*"question" + 0.002*"view" + '
  '0.002*"sentence" + 0.002*"good" + 0.002*"whole" + 0.002*"notice" + '
  '0.002*"know" + 0.002*"instead"'),
 (2,
  '0.016*"name" + 0.010*"spark" + 0.006*"memory" + 0.006*"value" + '
  '0.006*"localhost" + 0.006*"yarn" + 0.005*"hadoop" + 0.005*"resource" + '
  '0.004*"address" + 0.004*"scheduler"'),
 (3,
  '0.007*"library" + 0.007*"rapidminer" + 0.005*"load" + 0.005*"program_file" '
  '+ 0.005*"help" + 0.005*"try" + 0.005*"jri" + 0.005*"info" + 0.004*"r_home" '
  '+ 0.004*"window"'),
 (4,
  '0.003*"xpath" + 0.003*"extract" + 0.002*"right" + 0.002*"website" + '
  '0.002*"reviewcentre" + 0.002*"dont_know" + 0.002*"many" + 0.002*"day" + '
  '0.002*"appear" + 0.002*"review"'),
 (5,
  '0.024*"column" + 0.020*"datum" + 0.011*"use" + 0.008*"text" +

In [12]:
num_topics = 10
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicRapidMiner.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0   9594503  RapidMiner error: Regular Attributes must be o...   
1  10416219                           rapidminer some concepts   
2  10662637  How integrate RapidMiner in a PHP web applicat...   
3  14058589  Add a constant value to a numerical attribute ...   
4  14421563                        Read MYSQL db in RapidMiner   

                                                Body  RatingsSentiCR  \
0  I am attempting to learn to use RapidMiner, an...              -1   
1  In rapidminer what is the meaning of:\n\n    -...               1   
2  I've got an implementation in RapidMiner that ...               1   
3  I am working with rapidminer , I have a datase...               0   
4  I'm attempting to use the 'Read Database' oper...              -1   

   RatingsGPT35  RatingsGPTFineTuned  \
0            -1                 -1.0   
1            -1                 -1.0   
2            -1                 -1.0   
3         